In [6]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import datetime
import re
import json

In [7]:
with open("login.json", "r") as j:
    misDatos = json.load(j)

In [8]:
#para smart query
""" sf = simple_salesforce.Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
 """
#para lo demas
sf = Salesforce(
    username=misDatos["username"],
    password=misDatos["password"],
    security_token=misDatos["security_token"],
)
# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [9]:
resultadoAmsa = sf.query_all(
        """
        SELECT CALENDAR_YEAR(CreatedDate), COUNT(Id),CALENDAR_MONTH(CreatedDate)
    from Paciente_Programas__c
    Where Programas_por_Empresas__r.Holding__c like '%antofagasta minerals%'
    GROUP BY CALENDAR_YEAR(CreatedDate), CALENDAR_MONTH(CreatedDate)
    ORDER BY CALENDAR_MONTH(CreatedDate)
        """
)
amsaIngreso = pd.DataFrame(resultadoAmsa["records"])

amsaIngreso = amsaIngreso[
    [
        "expr0",
        "expr2",
        "expr1",
        
        
    ]
]
amsaIngreso = amsaIngreso.rename(
    columns={
        "expr0": "Año",
        "expr1": "Cantidad",
        "expr2": "Mes",
        
    }
)

meses = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}

amsaIngreso.replace({"Mes": meses}, inplace=True)

amsaIngreso.to_excel("F:/Reportes SF/Zurich/Cantida_AMSA.xlsx", index=False)
amsaIngreso.head(12)

,Año,Mes,Cantidad
0,2023,Enero,4091
1,2023,Febrero,267
2,2023,Marzo,195
3,2023,Mayo,3150
4,2023,Junio,346
5,2023,Julio,125
6,2023,Agosto,1
7,2023,Septiembre,174
8,2023,Octubre,199
9,2022,Noviembre,3


In [10]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape


(7789, 8)

In [11]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape


(7479, 5)

In [12]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c": "Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
    ]
]

Programas_empresas.shape

(7795, 10)

In [14]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx", index=False)

fin empresas y comienzo de rescate de pacientes


In [15]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape


(1857804, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


In [17]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape


(1473449, 4)

In [18]:
# Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

(1473449, 13)

In [19]:
empresas_unidas.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [20]:
# pacientes_empresas22 = pd.merge(
# left=Pacientes_SF,
# right=empresas_unidas,
# how="left",
# left_on="Id Account Paciente",
# right_on="Id Account",
# )
# pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
# pacientes_empresas22 = pacientes_empresas22[
# ["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
# ]

In [21]:
# pacientes_empresas22.shape

In [22]:
# apprix_1 = pacientes_empresas22.iloc[:848826,:]

# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)

In [23]:
# apprix_2 = pacientes_empresas22.iloc[848827:,:]#
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [24]:
# pacientes_empresas22.head()

Carga de Archivo carga


In [25]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "MetLife Rentas Vitalicias"
Pais = "Chile"
#Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion

In [26]:
Pacientes_Archivo.head(10)


,poliza,nombres,PATERNO,MATERNO,apellidos,RUTASEGURADO,RUTASEGURADODV,RUTBENEFICIARIO,RUTBENEFICIARIODV,rut1,RELACION,direccion,comuna,ciudad,FINGRESO,celular,email,FECHANACIMIENTO,nacimiento,sexo
0,340024125,RICARDO,INOSTROZA,TRIGO,INOSTROZA TRIGO,5901141,3,5901141,3,5901141-3,AS,ALCALDE BARTOLOME OYANEDEL 3447 VILLA LOS VOLV...,CALAMA,CALAMA,17/10/2023,963685946.0,ricardoinostroza498@gmail.com,25/10/1947,1947-10-25,M
1,340024125,IRMA DEL CARMEN,MORALES,SALAZAR,MORALES SALAZAR,5901141,3,7267661,0,7267661-0,CO,NaN,NaN,NaN,17/10/2023,NaN,NaN,10/06/1948,1948-06-10,F
2,340024125,JOSE MIGUEL,ARAVENA,GUTIERREZ,ARAVENA GUTIERREZ,3882278,0,3882278,0,3882278-0,AS,PASAJE EL ALICANTO 629 VILLA LAS LEYENDAS,CALAMA,CALAMA,11/10/2023,961584230.0,jaravena7g@gmail.com,07/12/1943,1943-12-07,M
3,340024125,SILVIA MARIA,ROJAS,ARIAS,ROJAS ARIAS,3882278,0,6859678,5,6859678-5,CO,NaN,NaN,NaN,11/10/2023,NaN,NaN,25/10/1951,1951-10-25,F
4,340024125,EDMUNDO ARMANDO,MARTINEZ,BAEZA,MARTINEZ BAEZA,5188550,3,5188550,3,5188550-3,AS,JULIO PRADO 1810 DEPTO 306,NUNOA,NUNOA,11/10/2023,939649592.0,eamb2046@gmail.com,20/05/1946,1946-05-20,M
5,340024125,MARTA DEL CARMEN,VERGARA,ANDRADE,VERGARA ANDRADE,5208754,6,5208754,6,5208754-6,AS,AV ZAPADORES 574,RECOLETA,RECOLETA,11/10/2023,997583268.0,NaN,29/09/1945,1945-09-29,F
6,340024125,BLANCA ELCIRA,GUIÑEZ,BARRA,GUIÑEZ BARRA,5510422,0,5510422,0,5510422-0,AS,UNO ORIENTE 1335 EDIFICIO LAS PALMAS DEPTO 72,TALCA,TALCA,11/10/2023,941265922.0,elciraguinez@gmail.com,20/06/1948,1948-06-20,F
7,340024125,SANTIAGO ADOLFO,PIZARRO,ELIZALDE,PIZARRO ELIZALDE,6566281,7,6566281,7,6566281-7,AS,ANTONINO TORO 976,ANTOFAGASTA,ANTOFAGASTA,11/10/2023,961995010.0,santiagopizarro146@gmail.com,09/07/1954,1954-07-09,M
8,340024125,LEONOR DEL CARMEN,ABARCA,MARAMBIO,ABARCA MARAMBIO,6775232,5,6775232,5,6775232-5,AS,PARQUE PUYEHUE OREINTE 2910,PENALOLEN,PENALOLEN,11/10/2023,935110343.0,leoabarca60@gmail.com,12/11/1952,1952-11-12,F
9,340024125,YOLANDA VICTORIA,FLORES,ANAYA,FLORES ANAYA,7101207,7,7101207,7,7101207-7,AS,COSTA RICA 9337 CASA C,LA FLORIDA,LA FLORIDA,11/10/2023,997562297.0,yoly.flores.a@gmail.com,20/04/1954,1954-04-20,F


In [27]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)

In [28]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()


,poliza,nombres,PATERNO,MATERNO,apellidos,RUTASEGURADO,RUTASEGURADODV,RUTBENEFICIARIO,RUTBENEFICIARIODV,rut1,...,direccion,comuna,ciudad,FINGRESO,celular,email,FECHANACIMIENTO,nacimiento,sexo,rut
0,340024125,RICARDO,INOSTROZA,TRIGO,INOSTROZA TRIGO,5901141,3,5901141,3,5901141-3,...,ALCALDE BARTOLOME OYANEDEL 3447 VILLA LOS VOLV...,CALAMA,CALAMA,17/10/2023,963685946.0,ricardoinostroza498@gmail.com,25/10/1947,1947-10-25,M,59011413
1,340024125,IRMA DEL CARMEN,MORALES,SALAZAR,MORALES SALAZAR,5901141,3,7267661,0,7267661-0,...,NaN,NaN,NaN,17/10/2023,NaN,NaN,10/06/1948,1948-06-10,F,72676610
2,340024125,JOSE MIGUEL,ARAVENA,GUTIERREZ,ARAVENA GUTIERREZ,3882278,0,3882278,0,3882278-0,...,PASAJE EL ALICANTO 629 VILLA LAS LEYENDAS,CALAMA,CALAMA,11/10/2023,961584230.0,jaravena7g@gmail.com,07/12/1943,1943-12-07,M,38822780
3,340024125,SILVIA MARIA,ROJAS,ARIAS,ROJAS ARIAS,3882278,0,6859678,5,6859678-5,...,NaN,NaN,NaN,11/10/2023,NaN,NaN,25/10/1951,1951-10-25,F,68596785
4,340024125,EDMUNDO ARMANDO,MARTINEZ,BAEZA,MARTINEZ BAEZA,5188550,3,5188550,3,5188550-3,...,JULIO PRADO 1810 DEPTO 306,NUNOA,NUNOA,11/10/2023,939649592.0,eamb2046@gmail.com,20/05/1946,1946-05-20,M,51885503


In [29]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12024\3578335771.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,poliza,nombres,PATERNO,MATERNO,apellidos,RUTASEGURADO,RUTASEGURADODV,RUTBENEFICIARIO,RUTBENEFICIARIODV,rut1,...,direccion,comuna,ciudad,FINGRESO,celular,email,FECHANACIMIENTO,nacimiento,sexo,rut
0,340024125,RICARDO,INOSTROZA,TRIGO,INOSTROZA TRIGO,5901141,3,5901141,3,5901141-3,...,ALCALDE BARTOLOME OYANEDEL 3447 VILLA LOS VOLV...,CALAMA,CALAMA,17/10/2023,963685946.0,ricardoinostroza498@gmail.com,25/10/1947,1947-10-25,M,59011413
1,340024125,IRMA DEL CARMEN,MORALES,SALAZAR,MORALES SALAZAR,5901141,3,7267661,0,7267661-0,...,NaN,NaN,NaN,17/10/2023,NaN,NaN,10/06/1948,1948-06-10,F,72676610
2,340024125,JOSE MIGUEL,ARAVENA,GUTIERREZ,ARAVENA GUTIERREZ,3882278,0,3882278,0,3882278-0,...,PASAJE EL ALICANTO 629 VILLA LAS LEYENDAS,CALAMA,CALAMA,11/10/2023,961584230.0,jaravena7g@gmail.com,07/12/1943,1943-12-07,M,38822780
3,340024125,SILVIA MARIA,ROJAS,ARIAS,ROJAS ARIAS,3882278,0,6859678,5,6859678-5,...,NaN,NaN,NaN,11/10/2023,NaN,NaN,25/10/1951,1951-10-25,F,68596785
4,340024125,EDMUNDO ARMANDO,MARTINEZ,BAEZA,MARTINEZ BAEZA,5188550,3,5188550,3,5188550-3,...,JULIO PRADO 1810 DEPTO 306,NUNOA,NUNOA,11/10/2023,939649592.0,eamb2046@gmail.com,20/05/1946,1946-05-20,M,51885503


In [30]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
    # Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12024\2830316430.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12024\2830316430.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [31]:
# aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(41, 31)

In [32]:
Pacientes_con_id.head()


,poliza,nombres,PATERNO,MATERNO,apellidos,RUTASEGURADO,RUTASEGURADODV,RUTBENEFICIARIO,RUTBENEFICIARIODV,rut1,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,340024125,Ricardo,INOSTROZA,TRIGO,Inostroza Trigo,5901141,3,5901141,3,5901141-3,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,340024125,Irma Del Carmen,MORALES,SALAZAR,Morales Salazar,5901141,3,7267661,0,7267661-0,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,340024125,Jose Miguel,ARAVENA,GUTIERREZ,Aravena Gutierrez,3882278,0,3882278,0,3882278-0,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,340024125,Silvia Maria,ROJAS,ARIAS,Rojas Arias,3882278,0,6859678,5,6859678-5,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,340024125,Edmundo Armando,MARTINEZ,BAEZA,Martinez Baeza,5188550,3,5188550,3,5188550-3,...,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")
def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""

In [34]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (Pacientes_con_id["email"].astype(str).str.lower())
    Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].apply(lambda y: isValid(y))
    #Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].str.replace("no informado", "")
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon":"Hombre"
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.lower().replace("nan", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("_", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("/", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_12024\1806998647.py:85: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")


,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonEmail,PersonBirthdate,HealthCloudGA__Gender__pc,BillingStreet,BillingState,BillingCity,Phone,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
0,False,Ricardo,Inostroza Trigo,59011413,ricardoinostroza498@gmail.com,1947-10-25,Hombre,alcalde bartolome oyanedel 3447 villa los volv...,Calama,Calama,963685946.,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
1,False,Irma Del Carmen,Morales Salazar,72676610,,1948-06-10,Mujer,NaN,NaN,NaN,,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
2,False,Jose Miguel,Aravena Gutierrez,38822780,jaravena7g@gmail.com,1943-12-07,Hombre,pasaje el alicanto 629 villa las leyendas,Calama,Calama,96158423.,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
3,False,Silvia Maria,Rojas Arias,68596785,,1951-10-25,Mujer,NaN,NaN,NaN,,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
4,False,Edmundo Armando,Martinez Baeza,51885503,eamb2046@gmail.com,1946-05-20,Hombre,julio prado 1810 depto 306,Nunoa,Nunoa,939649592.,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [35]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña
""" cha_arc = datetime.now()
dia_hoy = str(fecha_arc.day)
mes_hoy = str(fecha_arc.month)
 """


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "Sura": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
# clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)
""" clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)
 """

' clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)\n '

In [36]:
""" print(dia_hoy) """


' print(dia_hoy) '

In [37]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A


In [38]:
campaña_seleccionada.shape


(0, 13)

In [39]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )


In [40]:
if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    )


In [41]:
# apprix_1 = Pacientes_SF.iloc[:790552,:]
# apprix_2 = Pacientes_SF.iloc[790553:,:]
# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)

In [42]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)
